In [ ]:
def kernel_function_poly6(pt_a, pt_b, h):
    mag_squared = (pt_a[0] - pt_b[0])**2 + (pt_a[1] - pt_b[1])**2

    if mag_squared > h**2:
        return 0.0

    const_2d = 4.0 / (math.pi * h**8)
    return const_2d * (h**2 - mag_squared)**3


def kernel_function_spiky(pt_a, pt_b, h):
    dx = pt_a[0] - pt_b[0]
    dy = pt_a[1] - pt_b[1]
    mag_squared = dx*dx + dy*dy
    
    if mag_squared > h*h:
        return 0.0

    mag = math.sqrt(mag_squared)
    k = 10.0 / (math.pi * h**5)

    return k * (h - mag)**3

import math
def kernel_function_gradient_spiky(pt_a, pt_b, h):
    dx = pt_a[0] - pt_b[0]
    dy = pt_a[1] - pt_b[1]
    mag_squared = dx*dx + dy*dy

    # Return zero vector if outside kernel
    if mag_squared > h*h:
        return [0.0, 0.0]

    mag = math.sqrt(mag_squared)
    # Avoid division by zero if particles are at the same spot
    if mag == 0.0:
        return [0.0, 0.0]
    
    # 2D Spiky gradient constant
    c = -30.0 / (math.pi * h**5)

    # Multiply by ((h - r)² / r)
    factor = c * (h - mag)**2 / mag

    return [factor * dx, factor * dy]

def kernel_function_viscosity(pt_a, pt_b, h):
    dx = pt_a[0] - pt_b[0]
    dy = pt_a[1] - pt_b[1]
    mag_squared = dx*dx + dy*dy

    if mag_squared > h*h:
        return 0.0
    
    mag = math.sqrt(mag_squared)
    if mag == 0.0:
        return 0.0

    multiplier = 10.0 / (3.0 * math.pi * h**5)

    cube   = -mag**3 / (2.0 * h**3)
    square =  mag_squared / (h*h)
    linear =  (h / (2.0 * mag)) - 1.0

    return multiplier * (cube + square + linear)

def kernel_function_viscosity_laplacian(pt_a, pt_b, h):
    dx = pt_a[0] - pt_b[0]
    dy = pt_a[1] - pt_b[1]
    r2 = dx*dx + dy*dy

    # Outside the kernel radius => Laplacian is zero
    if r2 > h*h:
        return 0.0

    # This is basically a cone of radius h and height that makes the volume = 1
    # Volume of a cone is 1/3 * pi * r^2 * h
    # We want the volume to be 1, so we set h = 3 / (pi * r^2)
    # The height of the cone is 3 / (pi * r^2)
    # The volume of the cone is 1

    mag = math.sqrt(r2)
    
    if mag == 0.0:
        return 0.0
    
    height_proportion = 1 - (mag / h)
    volume = 3 / (math.pi * h ** 2)
    return volume * height_proportion


import numpy as np

xs = np.linspace(-1.25, 1.25, 1_000)
h = 1.0

outputs1 = [kernel_function_poly6([x, 0], [0, 0], h) for x in xs]
# outputs2 = [kernel_function_grad_poly6([x, 0], [0, 0], h)[0] for x in xs]
# outputs3 = [kernel_function_laplacian_poly6([x, 0], [0, 0], h) for x in xs]

# outputs4 = [kernel_function_spiky([x, 0], [0, 0], h) for x in xs]
outputs5 = [kernel_function_gradient_spiky([x, 0], [0, 0], h)[0] for x in xs]

# outputs7 = [kernel_function_viscosity([x, 0], [0, 0], h) for x in xs]
outputs9 = [kernel_function_viscosity_laplacian([x, 0], [0, 0], h) for x in xs]

import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 3, figsize=(12, 4))
axs[0].plot(xs, outputs1, label="Kernel Function")
# axs[0].plot(xs, outputs2, label="Kernel Gradient")
# axs[0].plot(xs, outputs3, label="Kernel Laplacian")
axs[0].set_title("Density Kernel Function")
axs[0].set_xlabel("x")
axs[0].set_ylabel("Kernel Value")

# axs[1].plot(xs, outputs4, label="Kernel Function")
axs[1].plot(xs, outputs5, label="Kernel Gradient")
axs[1].set_title("Kernel Function")
axs[1].set_xlabel("x")
axs[1].set_ylabel("Kernel Value")


# axs[2].plot(xs, outputs7, label="Kernel Viscosity")
axs[2].plot(xs, outputs9, label="Kernel Viscosity Laplacian")
axs[2].set_title("Kernel Viscosity")
axs[2].set_xlabel("x")
axs[2].set_ylabel("Kernel Value")

In [ ]:
# Integration check
EXTENT = 1.25
CELLS = 500

grid = np.linspace(-EXTENT, EXTENT, CELLS)

integral1 = 0.0
integral2 = 0.0
integral3 = 0.0
for x in grid:
    for y in grid:
        integral1 += kernel_function_poly6([x, y], [0, 0], 1.0) * (2 * EXTENT / CELLS) ** 2
        integral2 += abs(kernel_function_gradient_spiky([x, y], [0, 0], 1.0)[0]) * (2 * EXTENT / CELLS) ** 2
        integral3 += kernel_function_viscosity_laplacian([x, y], [0, 0], 1.0) * (2 * EXTENT / CELLS) ** 2

print(f"Integral of kernel function: {integral1}")
print(f"Integral of kernel function: {integral2}")
print(f"Integral of kernel function: {integral3}")

In [ ]:
# Constants
DOMAIN_SIZE = 60.0
MASS = 1.0
PRESSURE_MULTIPLIER = 1000.0
REST_DENSITY = 0.05
KERNEL_RADIUS = 5.0
VISCOSITY = 1.0
BOUNDS_FACTOR = -0.6

TIME_STEP = 0.01
TIME_STEPS = 5_000

DOMAIN_XLIM = [KERNEL_RADIUS, DOMAIN_SIZE - KERNEL_RADIUS]
DOMAIN_YLIM = [KERNEL_RADIUS, DOMAIN_SIZE - KERNEL_RADIUS]

GRAVITY_FORCE = 1.0

class Particle:
    def __init__(self, position, velocity):
        self.radius = 1 # TOOD: Make this a parameter
        self.position = position
        self.velocity = velocity
        self.density = 0.0
        self.pressure = 0.0
        self.pressure_force = [0.0, 0.0]
        self.viscosity_force = [0.0, 0.0]

class World:
    def __init__(self):
        self.particles = []
        self.time = 0.0

    def add_particle(self, position, velocity):
        self.particles.append(Particle(position, velocity))

    def recompute_buckets(self, search_radius):
        # Create buckets across the domain of size search_radius
        self.buckets = np.zeros((int(DOMAIN_SIZE / search_radius), int(DOMAIN_SIZE / search_radius)), dtype=object)
        # Clear buckets
        for i in range(self.buckets.shape[0]):
            for j in range(self.buckets.shape[1]):
                self.buckets[i, j] = list()

        # Add particles to buckets  
        for particle in self.particles:
            # Get the bucket index for the particle
            x_idx = int(particle.position[0] // search_radius)
            y_idx = int(particle.position[1] // search_radius)

            # Add particle to bucket
            self.buckets[x_idx, y_idx].append(particle)

    def get_candidate_neighbors(self, center, radius):
        # Get bucket
        x_idx = int(center[0] // radius)
        y_idx = int(center[1] // radius)
        # Get all particles in the bucket and those around it in 3x3 grid
        candidate_neighbors = []
        for i in range(x_idx - 1, x_idx + 2):
            for j in range(y_idx - 1, y_idx + 2):
                if i < 0 or i >= self.buckets.shape[0] or j < 0 or j >= self.buckets.shape[1]:
                    continue
                candidate_neighbors.extend(self.buckets[i, j])

        return candidate_neighbors
    
    def get_density_at_point(self, point, candidate_neighbors):
        density = 0.0
        for particle in candidate_neighbors:
            r = math.sqrt((point[0] - particle.position[0])**2 + (point[1] - particle.position[1])**2)
            if r > KERNEL_RADIUS:
                continue

            density += kernel_function_poly6(point, particle.position, KERNEL_RADIUS)

        return density

    def compute_forces(self):
        self.recompute_buckets(KERNEL_RADIUS)
        
        # Compute density for each particle
        for i, particle in enumerate(self.particles):
            candidate_neighbors = self.get_candidate_neighbors(particle.position, KERNEL_RADIUS)
            particle.density = self.get_density_at_point(particle.position, candidate_neighbors)

            # Compute pressure using equation of state
            particle.pressure = PRESSURE_MULTIPLIER * (particle.density - REST_DENSITY)

        for i, particle in enumerate(self.particles):
            candidate_neighbors = self.get_candidate_neighbors(particle.position, KERNEL_RADIUS)

            # Compute pressure force
            pressure_force = [0.0, 0.0]
            viscosity_force = [0.0, 0.0]
            for j, neighbor in enumerate(candidate_neighbors):
                if i == j:
                    continue

                pressure_force_mult = kernel_function_gradient_spiky(particle.position, neighbor.position, KERNEL_RADIUS)
                pressure_force[0] += - pressure_force_mult[0] * (particle.pressure + neighbor.pressure) / (2 * neighbor.density)
                pressure_force[1] += - pressure_force_mult[1] * (particle.pressure + neighbor.pressure) / (2 * neighbor.density)
                
                # Compute viscosity force
                viscosity_force_mult = kernel_function_viscosity_laplacian(particle.position, neighbor.position, KERNEL_RADIUS)
                viscosity_force[0] += VISCOSITY * viscosity_force_mult * (neighbor.velocity[0] - particle.velocity[0]) / neighbor.density
                viscosity_force[1] += VISCOSITY * viscosity_force_mult * (neighbor.velocity[1] - particle.velocity[1]) / neighbor.density

            particle.pressure_force = pressure_force
            particle.viscosity_force = viscosity_force

        for i, particle in enumerate(self.particles):
            # Apply gravity and comptute acceleration
            net_force  = [0.0, -GRAVITY_FORCE]
            net_force[0] += particle.pressure_force[0]
            net_force[1] += particle.pressure_force[1]
            net_force[0] += particle.viscosity_force[0]
            net_force[1] += particle.viscosity_force[1]

            acceleration = [net_force[0] / particle.density, net_force[1] / particle.density] # F = ma | a = F/m
            particle.velocity[0] += acceleration[0] * TIME_STEP
            particle.velocity[1] += acceleration[1] * TIME_STEP

            # Update position
            particle.position[0] += particle.velocity[0] * TIME_STEP
            particle.position[1] += particle.velocity[1] * TIME_STEP

            # Handle boundary conditions
            if particle.position[0] < DOMAIN_XLIM[0]:
                particle.position[0] = DOMAIN_XLIM[0] + (DOMAIN_XLIM[0] - particle.position[0])
                particle.velocity[0] *= BOUNDS_FACTOR
            elif particle.position[0] > DOMAIN_XLIM[1]:
                particle.position[0] = DOMAIN_XLIM[1] - (particle.position[0] - DOMAIN_XLIM[1])
                particle.velocity[0] *= BOUNDS_FACTOR
            if particle.position[1] < DOMAIN_YLIM[0]:
                particle.position[1] = DOMAIN_YLIM[0] + (DOMAIN_YLIM[0] - particle.position[1])
                particle.velocity[1] *= BOUNDS_FACTOR
            elif particle.position[1] > DOMAIN_YLIM[1]:
                particle.position[1] = DOMAIN_YLIM[1] - (particle.position[1] - DOMAIN_YLIM[1])
                particle.velocity[1] *= BOUNDS_FACTOR

In [ ]:
import random
import matplotlib.pyplot as plt
from tqdm import tqdm

# Create a world and add particles
world = World()
frame_num = 0
# Every 50 time steps, spawn a new set of 10 particles
for timestep in tqdm(range(TIME_STEPS)):
    if timestep % 1200 == 0 and timestep <= 1200 * 2:
        for i in range(50):
            x = random.uniform(DOMAIN_XLIM[0], DOMAIN_XLIM[1])
            y = random.uniform((DOMAIN_YLIM[0] + DOMAIN_YLIM[1]) / 2, DOMAIN_YLIM[1])
            world.add_particle([x, y], [0.0, 0.0])

    # Compute forces and update particles
    world.compute_forces()
    world.time += TIME_STEP

    # Only render every 10 time steps (0.1 seconds)
    if timestep % 10 == 0:
        fig, axs = plt.subplots(2,2, figsize=(12,12))
        fig.suptitle(f"Time: {world.time:.2f} seconds", fontsize=16)
        axs[0,0].set_title("Density")
        axs[0,1].set_title("Particle Velocity")
        axs[1,0].set_title("Pressure")
        axs[1,1].set_title("Forces")
        plt.tight_layout()
        for ax in axs.flatten():
            ax.set_aspect('equal')
            ax.set_xlim(0, DOMAIN_SIZE)
            ax.set_ylim(0, DOMAIN_SIZE)
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_xlabel('x')
            ax.set_ylabel('y')

        # Compute density at each location
        DENSITY_RESOLUTION = 200
        density_map = np.zeros((DENSITY_RESOLUTION, DENSITY_RESOLUTION))
        for i in range(DENSITY_RESOLUTION):
            for j in range(DENSITY_RESOLUTION):
                x = i * (DOMAIN_SIZE / DENSITY_RESOLUTION)
                y = j * (DOMAIN_SIZE / DENSITY_RESOLUTION)
                density_map[i][j] = world.get_density_at_point([x, y], world.particles)

        density_map = density_map.T

        axs[0,0].imshow(density_map, extent=[0, DOMAIN_SIZE, 0, DOMAIN_SIZE], vmin=0, vmax=0.04, origin='lower', cmap='Blues')

        # cbar = plt.colorbar(axs[0,1].imshow(density_map, extent=[0, DOMAIN_SIZE, 0, DOMAIN_SIZE], origin='lower', cmap='hot', vmin=0.0, vmax=0.15), ax=axs[0,1])
        # cbar.set_label('Density', rotation=270, labelpad=15)

        cbar = plt.colorbar(axs[1,0].imshow(density_map, extent=[0, DOMAIN_SIZE, 0, DOMAIN_SIZE], origin='lower', cmap='hot', vmin=0.0, vmax=0.15), ax=axs[1,0], fraction=0.046, pad=0.02)


        # Get vectors for forces
        pressure_vectors = []
        viscosity_vectors = []
        gravity_vectors = []
        velocity_vectors = []
        max_magnitude = 0.0
        for particle in world.particles:
            pressure_vectors.append([particle.position[0], particle.position[1], particle.pressure_force[0], particle.pressure_force[1]])
            viscosity_vectors.append([particle.position[0], particle.position[1], particle.viscosity_force[0], particle.viscosity_force[1]])
            gravity_vectors.append([particle.position[0], particle.position[1], 0.0, -GRAVITY_FORCE])
            magnitude1 = math.sqrt(particle.pressure_force[0]**2 + particle.pressure_force[1]**2)
            magnitude2 = math.sqrt(particle.viscosity_force[0]**2 + particle.viscosity_force[1]**2)
            max_magnitude = max(max_magnitude, magnitude1, magnitude2, GRAVITY_FORCE)

            velocity_vectors.append([particle.position[0], particle.position[1], particle.velocity[0], particle.velocity[1]])

        # Plot particles
        for i, particle in enumerate(world.particles):
            MAX_ARROW_LENGTH = 10
            circle = plt.Circle(particle.position, particle.radius, color='blue', fill=True)
            axs[0,1].add_artist(circle)
            # Plot velocity vectors
            axs[0,1].arrow(particle.position[0], particle.position[1], particle.velocity[0] * 0.1, particle.velocity[1] * 0.1, color='black', head_width=0.5, head_length=0.5)
            # Plot density circles

            circle = plt.Circle(particle.position, particle.radius, color='black', fill=False)
            axs[1,1].add_artist(circle)

            # Plot pressure force vectors
            axs[1,1].arrow(particle.position[0], particle.position[1], MAX_ARROW_LENGTH * particle.pressure_force[0] / max_magnitude, MAX_ARROW_LENGTH * particle.pressure_force[1] / max_magnitude, color='red', head_width=0.5, head_length=0.5)
            # Plot viscosity force vectors
            axs[1,1].arrow(particle.position[0], particle.position[1], MAX_ARROW_LENGTH * particle.viscosity_force[0] / max_magnitude, MAX_ARROW_LENGTH * particle.viscosity_force[1] / max_magnitude, color='green', head_width=0.5, head_length=0.5)
            # Plot gravity force vectors
            axs[1,1].arrow(particle.position[0], particle.position[1], MAX_ARROW_LENGTH * 0.0 / max_magnitude, MAX_ARROW_LENGTH * -GRAVITY_FORCE / max_magnitude, color='blue', head_width=0.5, head_length=0.5)


        # Save figure to file
        frame_num += 1
        plt.savefig(f"output/frame_{frame_num:04d}.png")
        plt.close(fig)



In [ ]:
!ffmpeg -framerate 10 -y -i output/frame_%04d.png -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p output/output.mp4